In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks

os.chdir('./Comp7103_Assignment1_data_mining_and_NLP')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [2]:
!git clone https://github.com/google-research/bert.git
!pip install bert-serving-server # server
!pip install bert-serving-client # client, independent of 'bert-serving-server'
!pip install transformers

fatal: destination path 'bert' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=92832e6a8d1cdccff150a6f7ccad1fd250b10c94a67ea3c068c75bb5df17067c
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built GPUtil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.4 

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras import optimizers

import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score

import random
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, RobertaConfig
from transformers import pipeline
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import torchsummary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
df = pd.read_csv("amazon/Reviews.csv")
df = df.dropna()
print(len(df))
df = df.sample(n=int(len(df)*0.02))
print(len(df))

568411
11368


In [7]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
257877,257878,B00125ZCWI,A2GN22343BJY3S,issi27,2,3,5,1301443200,Keeps my mother-in-law regular,This is a great product. I send it to my mothe...
141069,141070,B001M050T8,AJ1J7BSBN5SY4,James M. Bennett,19,20,1,1258934400,New Ovaltine - Please give me the original for...,What a disappointment it was to try the new Ne...
337021,337022,B004CYLWGQ,A1PM0LSZTA99PE,Sara H.,6,7,2,1295049600,Not As Good As Before,Saw this in the grocery store recently and was...
500810,500811,B002TN49F8,AA98O5326L88T,"Aaron Silverman ""DJ Kuul A""",1,1,4,1295136000,"Smells great, with a nice subtle flavor",I really liked this one -- it has enough vanil...
536503,536504,B001EQ4SRU,A1Z26GXUS1DLEZ,"Dominic Ross ""coffee consumer""",0,1,2,1260144000,Better off purchasing from your local store,"Bought this last week, shipping was very fast...."


In [8]:
robert_df = df.loc[:,['Text', 'Summary', 'HelpfulnessNumerator','Score']]
robert_df['Score'] -= 1
train_df, test_df = train_test_split(robert_df, test_size=0.2, random_state=3074)
robert_df.head()

,Text,Summary,HelpfulnessNumerator,Score
257877,This is a great product. I send it to my mothe...,Keeps my mother-in-law regular,2,4
141069,What a disappointment it was to try the new Ne...,New Ovaltine - Please give me the original for...,19,0
337021,Saw this in the grocery store recently and was...,Not As Good As Before,6,1
500810,I really liked this one -- it has enough vanil...,"Smells great, with a nice subtle flavor",1,3
536503,"Bought this last week, shipping was very fast....",Better off purchasing from your local store,0,1


In [9]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5, output_hidden_states=True)
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 128)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [10]:
# Tokenize and prepare dataloader
encoded_data_train = tokenizer(list(train_df['Text']), padding = True, truncation=True)
input_ids_train = torch.tensor(np.array(encoded_data_train['input_ids']))
attention_masks_train = torch.tensor(np.array(encoded_data_train['attention_mask']))
labels_train = torch.tensor(train_df['Score'].to_numpy())
help_train = torch.tensor(train_df['HelpfulnessNumerator'].to_numpy())

encoded_data_test = tokenizer(list(test_df['Text']), padding = True, truncation=True)
input_ids_test = torch.tensor(np.array(encoded_data_test['input_ids']))
attention_masks_test = torch.tensor(np.array(encoded_data_test['attention_mask']))
labels_test = torch.tensor(test_df['Score'].to_numpy())
help_test = torch.tensor(test_df['HelpfulnessNumerator'].to_numpy())

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train, help_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test, help_test)

batch_size = 4

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=batch_size)

In [11]:
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)        
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
encoded_data_train[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [13]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':     batch[2],
                 }
        helpfulness = batch[3]

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

f1_scores = []
training_loss = []
validation_loss = []    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        helpfulness = batch[3]

        model = model.to(device)
        outputs = model(**inputs)
        # logits, gt = outputs[1].view(-1, 5), inputs['labels'].view(-1)
        # print(CrossEntropyLoss()(, ))
        loss = outputs[0] * (sum(helpfulness)/4+1)
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./bert_checkpopint/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    f1_scores.append(val_f1)
    validation_loss.append(val_loss)
    training_loss.append(loss_train_avg)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.485242730733122
Validation loss: 0.6457561014681688
F1 Score (Weighted): 0.742950486133296


Epoch 2:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 2
Training loss: 2.0742318785064193
Validation loss: 0.7970189459945987
F1 Score (Weighted): 0.7523303157782703


...


Epoch 8
Training loss: 0.5681560311198538
Validation loss: 2.0259647718355294
F1 Score (Weighted): 0.754573995565095


Epoch 9:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.35822243523633207
Validation loss: 2.2629607418359887
F1 Score (Weighted): 0.7468425904243865


Epoch 10:   0%|          | 0/2274 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.24771207982338203
Validation loss: 2.2495130551821245
F1 Score (Weighted): 0.7540073444400917
